In [1]:
!pip install torchsummary

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torch.nn import Module
from torch.nn import Sequential
from torch.nn import Conv2d
from torch.nn import Linear
from torch.nn import MaxPool2d
from torch.nn import AvgPool2d
from torch.nn import BatchNorm2d
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Tanh
from torch.nn import ELU
from torch.nn import LogSoftmax
from torch.nn import Dropout
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import flatten

from sklearn.metrics import accuracy_score, f1_score
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms.autoaugment import AutoAugmentPolicy
from torch.utils.data.sampler import SubsetRandomSampler
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor
from torchsummary import summary
import cv2
import glob
import pickle
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

cuda


In [8]:
from torchvision.models import resnet50

resNet=resnet50(pretrained=True).to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 151MB/s] 


# Feature Extraction

In [7]:
class FeatureExtractor(nn.Module):

  def __init__(self, model):
    super(FeatureExtractor, self).__init__()
    self.layer0=Sequential(
        model.conv1,
        model.bn1,
        model.relu,
        model.maxpool
    )
    self.layer1 = model.layer1
    self.layer2=model.layer2
    self.layer3=model.layer3
    self.layer4=model.layer4
    
    self.pooling = nn.AdaptiveAvgPool2d(output_size=(2, 2))
    
    self.flatten = nn.Flatten()
    
  
  def forward(self, x):
    out=self.layer0(x)
    out = self.layer1(out)
    out=self.layer2(out)
    out=self.layer3(out)
    out=self.layer4(out)
    out = self.pooling(out)
    out = self.flatten(out)
    return out 


In [20]:
transform = transforms.Compose([
    transforms.Resize((720, 720)),
    transforms.ToTensor()
])
train_data_path = '/kaggle/input/fruit-and-vegetable-image-recognition/train'

train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
train_loader.dataset

Dataset ImageFolder
    Number of datapoints: 3115
    Root location: /kaggle/input/fruit-and-vegetable-image-recognition/train
    StandardTransform
Transform: Compose(
               Resize(size=(720, 720), interpolation=bilinear, max_size=None, antialias=warn)
               ToTensor()
           )

In [21]:
test_data_path = '/kaggle/input/fruit-and-vegetable-image-recognition/test'
test_dataset = datasets.ImageFolder(root=test_data_path, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

# Training

In [9]:
class NewFeatureExtractor(FeatureExtractor):

    def __init__(self, model, num_classes):
        super(NewFeatureExtractor, self).__init__(model)
        # Add a fully connected layer
        self.fc = Sequential(
            nn.Linear(in_features=2048*2*2, out_features=2048),
            ReLU(),
            Dropout(0.3),
            nn.Linear(in_features=2048,out_features=512),
            ReLU(),
            Dropout(0.3),
            nn.Linear(in_features=512,out_features=num_classes)
        )

    def forward(self, x):
        # Call the forward method of the parent class
        out = super(NewFeatureExtractor, self).forward(x)
        # Pass the output through the fully connected layer
        out = self.fc(out)
        return out

In [14]:
newModel=NewFeatureExtractor(resNet,36).to(device)
num_epochs = 25
learning_rate=0.001
criterion = nn.CrossEntropyLoss()

checkpoint= torch.load('/kaggle/input/resnet/pytorch/updated/1/modelNew.pth')
newModel.load_state_dict(checkpoint)

optimizer = optim.Adam(newModel.parameters(), lr=learning_rate)
newModel.eval()
summary(newModel,(3,720,720))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 360, 360]           9,408
       BatchNorm2d-2         [-1, 64, 360, 360]             128
              ReLU-3         [-1, 64, 360, 360]               0
         MaxPool2d-4         [-1, 64, 180, 180]               0
            Conv2d-5         [-1, 64, 180, 180]           4,096
       BatchNorm2d-6         [-1, 64, 180, 180]             128
              ReLU-7         [-1, 64, 180, 180]               0
            Conv2d-8         [-1, 64, 180, 180]          36,864
       BatchNorm2d-9         [-1, 64, 180, 180]             128
             ReLU-10         [-1, 64, 180, 180]               0
           Conv2d-11        [-1, 256, 180, 180]          16,384
      BatchNorm2d-12        [-1, 256, 180, 180]             512
           Conv2d-13        [-1, 256, 180, 180]          16,384
      BatchNorm2d-14        [-1, 256, 1

In [ ]:
epoch_losses = []

for epoch in range(num_epochs):
    newModel.train()
    running_loss = 0.0
    with tqdm(train_loader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
        for images, labels in tepoch:
            images, labels = images.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = newModel(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            tepoch.set_postfix(loss=running_loss / ((tepoch.n + 1) * train_loader.batch_size))

    # Print statistics
    epoch_loss = running_loss / len(train_dataset)
    epoch_losses.append(epoch_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

  0%|          | 0/195 [00:00<?, ?batch/s]

In [ ]:
torch.save(newModel.state_dict(), 'modelNew.pth')

# Evaluation

In [10]:
def validate(model, dataloader, sanity_check=False):
    model.eval()
    total_loss = 0
    predictions = []
    truths = []
    loss_fn = torch.nn.CrossEntropyLoss()

    device = next(model.parameters()).device  # Get the device of the model's parameters
    
    with torch.no_grad():
        for batch in tqdm(dataloader):
            x, y = batch[0].to(device), batch[1].to(device)  # Move input data to the same device as model
            
            output = model(x)
            output = nn.Softmax(dim=-1)(output)
            loss = loss_fn(output, y)
            total_loss += loss.detach().cpu().item() / len(dataloader)
            
            preds = torch.argmax(output, dim=-1)
            predictions.extend(preds.cpu())
            truths.extend(y.cpu())
        
    acc = accuracy_score(y_true=truths, y_pred=predictions)
    f1 = f1_score(y_true=truths, y_pred=predictions, average='macro')
    
    return total_loss, acc, f1, predictions


In [13]:
scores = validate(newModel,test_loader)
print(f"| Test Loss: {scores[0]: 7.3f}  | Test acc: {scores[1]: 1.5f}  | Test f1: {scores[2]: 1.5f}  |")

  0%|          | 0/23 [00:00<?, ?it/s]

| Test Loss:   2.695  | Test acc:  0.96379  | Test f1:  0.96311  |


# Bike And Horse Dataset

In [11]:
feature_extractor = FeatureExtractor(resNet)
feature_extractor = feature_extractor.to(device)
summary(feature_extractor,(3,280,470))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 140, 235]           9,408
       BatchNorm2d-2         [-1, 64, 140, 235]             128
              ReLU-3         [-1, 64, 140, 235]               0
         MaxPool2d-4          [-1, 64, 70, 118]               0
            Conv2d-5          [-1, 64, 70, 118]           4,096
       BatchNorm2d-6          [-1, 64, 70, 118]             128
              ReLU-7          [-1, 64, 70, 118]               0
            Conv2d-8          [-1, 64, 70, 118]          36,864
       BatchNorm2d-9          [-1, 64, 70, 118]             128
             ReLU-10          [-1, 64, 70, 118]               0
           Conv2d-11         [-1, 256, 70, 118]          16,384
      BatchNorm2d-12         [-1, 256, 70, 118]             512
           Conv2d-13         [-1, 256, 70, 118]          16,384
      BatchNorm2d-14         [-1, 256, 

In [24]:
bikehorsemodel=NewFeatureExtractor(resNet,2).to(device)
summary(bikehorsemodel,(3,280,480))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 140, 240]           9,408
       BatchNorm2d-2         [-1, 64, 140, 240]             128
              ReLU-3         [-1, 64, 140, 240]               0
         MaxPool2d-4          [-1, 64, 70, 120]               0
            Conv2d-5          [-1, 64, 70, 120]           4,096
       BatchNorm2d-6          [-1, 64, 70, 120]             128
              ReLU-7          [-1, 64, 70, 120]               0
            Conv2d-8          [-1, 64, 70, 120]          36,864
       BatchNorm2d-9          [-1, 64, 70, 120]             128
             ReLU-10          [-1, 64, 70, 120]               0
           Conv2d-11         [-1, 256, 70, 120]          16,384
      BatchNorm2d-12         [-1, 256, 70, 120]             512
           Conv2d-13         [-1, 256, 70, 120]          16,384
      BatchNorm2d-14         [-1, 256, 

In [25]:
from torch.utils.data import random_split
transform1 = transforms.Compose([
    transforms.Resize((280, 480)),
    transforms.ToTensor()
])
train_data_path = '/kaggle/input/bikehorse/Assignment2_BikeHorses'

bikehorsedataset = datasets.ImageFolder(root=train_data_path, transform=transform1)

validation_ratio = 0.2  # 20% of the data will be used for validation

# Calculate the sizes of the train and validation sets
train_size = int((1 - validation_ratio) * len(bikehorsedataset))
val_size = len(bikehorsedataset) - train_size

# Split the dataset into train and validation sets
train_dataset, val_dataset = random_split(bikehorsedataset, [train_size, val_size])


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader=torch.utils.data.DataLoader(val_dataset,batch_size=4,shuffle=True)

train_loader.dataset

In [26]:
epoch_losses1 = []
num_epochs = 15
learning_rate=0.005
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(bikehorsemodel.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    bikehorsemodel.train()
    running_loss = 0.0
    with tqdm(train_loader, unit="batch") as tepoch:
        tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
        for images, labels in tepoch:
            images, labels = images.to(device), labels.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = bikehorsemodel(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            tepoch.set_postfix(loss=running_loss / ((tepoch.n + 1) * train_loader.batch_size))

    # Print statistics
    epoch_loss = running_loss / len(train_dataset)
    epoch_losses1.append(epoch_loss)
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')

  0%|          | 0/36 [00:00<?, ?batch/s]

Epoch [1/15], Loss: 0.7445


  0%|          | 0/36 [00:00<?, ?batch/s]

Epoch [2/15], Loss: 0.3502


  0%|          | 0/36 [00:00<?, ?batch/s]

...

Epoch [13/15], Loss: 0.1035


  0%|          | 0/36 [00:00<?, ?batch/s]

Epoch [14/15], Loss: 0.1378


  0%|          | 0/36 [00:00<?, ?batch/s]

Epoch [15/15], Loss: 0.0777


In [30]:
scores = validate(bikehorsemodel,test_loader)
print(f"| Test Loss: {scores[0]: 7.3f}  | Test acc: {scores[1]: 1.5f}  | Test f1: {scores[2]: 1.5f}  |")

  0%|          | 0/9 [00:00<?, ?it/s]

| Test Loss:   0.315  | Test acc:  1.00000  | Test f1:  1.00000  |
